In [2]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

parquet_file = "../../local.parquet"
table = pq.read_table(parquet_file)

male_columns = [col for col in table.column_names if "_m_" in col]
female_columns = [col for col in table.column_names if "_f_" in col]

def sum_column_with_filter(column):
    non_negative_column = pc.if_else(pc.less(column, 0), pa.array([0] * len(column)), column)
    return pc.sum(non_negative_column)

sum_pop_f_2020 = sum(sum_column_with_filter(table.column(col)) for col in female_columns)
sum_pop_m_2020 = sum(sum_column_with_filter(table.column(col)) for col in male_columns)
sum_pop_2020 = sum_pop_f_2020 + sum_pop_m_2020

new_table = table.append_column("sum_pop_f_2020", sum_pop_f_2020)
new_table = new_table.append_column("sum_pop_m_2020", sum_pop_m_2020)
new_table = new_table.append_column("sum_pop_2020", sum_pop_2020)

updated_parquet_file = "updated_local.parquet"
pq.write_table(new_table, updated_parquet_file)

FileNotFoundError: local.parquet